In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

import pickle

In [6]:
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('dev.csv')

In [7]:
notfake = train_data[train_data['label']== 1]
fake = train_data[train_data['label'] == 0]
true_word_list = []

In [8]:
for i in range(0,notfake['review'].count()):
    true_word_temp = notfake['review'].iloc[i].split()
    true_word_list.append(true_word_temp)
from itertools import chain
list1 = list(chain.from_iterable(true_word_list))
from collections import Counter 
true_mc = Counter(list1).most_common()
 
fake_word_list= fake['review'].str.split(expand=True).stack()
from collections import Counter 
fake_mc = Counter(fake_word_list).most_common()

In [9]:
df_true = pd.DataFrame(true_mc)
df_false = pd.DataFrame(fake_mc)
df_false1 = df_false[0:23]
df_true1 = df_true[0:23]
df_common = list(set(df_true1.iloc[:,0]) & set(df_false1.iloc[:,0]))
df_common

['of',
 'with',
 'it',
 'on',
 'had',
 'to',
 'is',
 'this',
 'for',
 'the',
 'we',
 'I',
 'my',
 'a',
 'and',
 'you',
 'in',
 'The',
 'but',
 'that',
 'was']

In [11]:
notfake

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,The food at snack is a selection of popular Gr...
1,1,924,0,3.0,1,2013-05-16,This little place in Soho is wonderful. I had ...
2,2,925,0,4.0,1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,3,926,0,4.0,1,2011-07-28,This is a beautiful quaint little restaurant o...
4,4,927,0,4.0,1,2010-11-01,Snack is great place for a casual sit down lu...
...,...,...,...,...,...,...,...
250677,358685,161043,921,5.0,1,2012-07-31,"This place is amazing, I loved it so much that..."
250678,358686,161044,921,5.0,1,2012-06-12,"I love this place. Drinks are great, atmospher..."
250679,358688,161046,921,5.0,1,2011-12-01,Bijan is the best bar I have been to in Brookl...
250680,358689,161047,921,5.0,1,2011-11-15,Great local lounge spot! After work and regula...


https://towardsdatascience.com/text-classification-using-k-nearest-neighbors-46fa8a77acc5

In [13]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to /Users/robin/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package wordnet to /Users/robin/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /Users/robin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/robin/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [28]:
class KNN_NLC_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in range(len(x_test)):
            max_sim = 0
            max_index = 0
            for j in range(self.x_train.shape[0]):
                temp = self.document_similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict

In [29]:
def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


def doc_to_synsets(self, doc):
        """
            Returns a list of synsets in document.
            Tokenizes and tags the words in the document doc.
            Then finds the first synset for each word/tag combination.
        If a synset is not found for that combination it is skipped.

        Args:
            doc: string to be converted

        Returns:
            list of synsets
        """
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  

In [30]:
 def similarity_score(self, s1, s2, distance_type = 'path'):
          """
          Calculate the normalized similarity score of s1 onto s2
          For each synset in s1, finds the synset in s2 with the largest similarity value.
          Sum of all of the largest similarity values and normalize this value by dividing it by the
          number of largest similarity values found.

          Args:
              s1, s2: list of synsets from doc_to_synsets

          Returns:
              normalized similarity score of s1 onto s2
          """
s1_largest_scores = []

for i, s1_synset in enumerate(s1, 0):
    max_score = 0
    for s2_synset in s2:
        if distance_type == 'path':
            score = s1_synset.path_similarity(s2_synset, simulate_root = False)
        else:
            score = s1_synset.wup_similarity(s2_synset)                  
        if score != None:
            if score > max_score:
                max_score = score

        if max_score != 0:
            s1_largest_scores.append(max_score)

mean_score = np.mean(s1_largest_scores)

return mean_score  

NameError: name 's1' is not defined

In [32]:
train = pd.read_csv(train_path)
validation = pd.read_csv(val_path)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cnt_vectorizer = CountVectorizer(stop_words='english', binary=True)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', binary=True)
cnt_vectorizer.fit(train['review'])
tfidf_vectorizer.fit(train['review'])

cnt_X_train = cnt_vectorizer.transform(train['review'])
tfidf_X_train = tfidf_vectorizer.transform(train['review'])

cnt_X_dev = cnt_vectorizer.transform(validation['review'])
tfidf_X_dev = tfidf_vectorizer.transform(validation['review'])

Y_train = train['label']
Y_dev = validation['label']

In [34]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(cnt_X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [36]:
y_pred = classifier.predict(cnt_X_train)

In [37]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [38]:
y_new_pred = classifier.predict(cnt_X_dev)

In [40]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_dev, y_new_pred))
print(classification_report(Y_dev, y_new_pred))

[[26678  5592]
 [ 2907   741]]
              precision    recall  f1-score   support

           0       0.90      0.83      0.86     32270
           1       0.12      0.20      0.15      3648

    accuracy                           0.76     35918
   macro avg       0.51      0.51      0.51     35918
weighted avg       0.82      0.76      0.79     35918



In [44]:
classifier10 = KNeighborsClassifier(n_neighbors=10)
classifier10.fit(cnt_X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [45]:
y_new10_pred = classifier10.predict(cnt_X_dev)

In [46]:
print(confusion_matrix(Y_dev, y_new10_pred))
print(classification_report(Y_dev, y_new10_pred))

[[29869  2401]
 [ 3291   357]]
              precision    recall  f1-score   support

           0       0.90      0.93      0.91     32270
           1       0.13      0.10      0.11      3648

    accuracy                           0.84     35918
   macro avg       0.52      0.51      0.51     35918
weighted avg       0.82      0.84      0.83     35918



In [47]:
classifier15 = KNeighborsClassifier(n_neighbors=15)
classifier15.fit(cnt_X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [48]:
y_new15_pred = classifier15.predict(cnt_X_dev)

In [49]:
print(confusion_matrix(Y_dev, y_new15_pred))
print(classification_report(Y_dev, y_new15_pred))

[[29347  2923]
 [ 3246   402]]
              precision    recall  f1-score   support

           0       0.90      0.91      0.90     32270
           1       0.12      0.11      0.12      3648

    accuracy                           0.83     35918
   macro avg       0.51      0.51      0.51     35918
weighted avg       0.82      0.83      0.82     35918



In [50]:
classifier20 = KNeighborsClassifier(n_neighbors=20)
classifier20.fit(cnt_X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='uniform')

In [51]:
y_new20_pred = classifier20.predict(cnt_X_dev)

In [52]:
print(confusion_matrix(Y_dev, y_new20_pred))
print(classification_report(Y_dev, y_new20_pred))

[[30589  1681]
 [ 3418   230]]
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     32270
           1       0.12      0.06      0.08      3648

    accuracy                           0.86     35918
   macro avg       0.51      0.51      0.50     35918
weighted avg       0.82      0.86      0.84     35918



In [53]:
classifier11 = KNeighborsClassifier(n_neighbors=11)
classifier11.fit(cnt_X_train, Y_train)
y_new11_pred = classifier11.predict(cnt_X_dev)
print(confusion_matrix(Y_dev, y_new11_pred))
print(classification_report(Y_dev, y_new11_pred))

[[28628  3642]
 [ 3156   492]]
              precision    recall  f1-score   support

           0       0.90      0.89      0.89     32270
           1       0.12      0.13      0.13      3648

    accuracy                           0.81     35918
   macro avg       0.51      0.51      0.51     35918
weighted avg       0.82      0.81      0.82     35918



In [54]:
classifier13 = KNeighborsClassifier(n_neighbors=13)
classifier13.fit(cnt_X_train, Y_train)
y_new13_pred = classifier13.predict(cnt_X_dev)
print(confusion_matrix(Y_dev, y_new13_pred))
print(classification_report(Y_dev, y_new13_pred))

[[29022  3248]
 [ 3203   445]]
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     32270
           1       0.12      0.12      0.12      3648

    accuracy                           0.82     35918
   macro avg       0.51      0.51      0.51     35918
weighted avg       0.82      0.82      0.82     35918



In [55]:
classifier14 = KNeighborsClassifier(n_neighbors=14)
classifier14.fit(cnt_X_train, Y_train)
y_new14_pred = classifier14.predict(cnt_X_dev)
print(confusion_matrix(Y_dev, y_new14_pred))
print(classification_report(Y_dev, y_new14_pred))

[[30212  2058]
 [ 3349   299]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     32270
           1       0.13      0.08      0.10      3648

    accuracy                           0.85     35918
   macro avg       0.51      0.51      0.51     35918
weighted avg       0.82      0.85      0.83     35918



In [42]:
#copied from am
import json
base = '../data/processed/dev/'
def writeJsonFile(fname, data,  base=base):
    with open(base + fname +'.json', 'w') as outfile:
        json.dump(data, outfile)
    print('Successfully written to {}'.format(fname))
    
def readJsonFile(fname, base=base):
    with open(base + fname + '.json', 'r') as f:
        data = json.load(f)
    return data

In [43]:
knn_model_params = {'neighbor':[], 'train_accuracy':[],'test_accuracy':[],'test_auc':[], 'test_ap':[]}
results_path = '../../data/processed/knn_unbalanced_results_drh382.csv'
x = pd.read_csv(results_path)
knn_model_params = x.to_dict(orient='list')

FileNotFoundError: [Errno 2] File b'../../data/processed/knn_unbalanced_results_drh382.csv' does not exist: b'../../data/processed/knn_unbalanced_results_drh382.csv'

In [ ]:
#copied from amelia
k = kernels[0]
print('Now trying: ' + k + ' kernels...')
for c in cs[2:]:
    t = Timer()
    t.start()
    svm_model_params['kernel'].append(k)
    svm_model_params['C'].append(c)
    params = {'kernel':k,'C':c, 'gamma':'scale', 'random_state': 519}
    fitted_model = fitted_svm(params)
    metrics = ClassifierMetrics(tfidf_X_train, Y_train, tfidf_X_dev, Y_dev, fitted_model)
    for m_k in metrics.keys():
        svm_model_params[m_k].append(metrics[m_k])
    elapsed = t.stop()
    svm_pd = pd.DataFrame(svm_model_params)
    results_path = '../../data/processed/svm_svc_tfidf_unbalanced_results_ac4119.csv'
    svm_pd.to_csv(results_path, sep=',')
    print('Elapsed time:',elapsed)